In [1]:
# ============================================================================
# CELL 1: Imports
# ============================================================================
import pandas as pd
import numpy as np
import sqlite3
import json
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, classification_report
import pickle
import optuna
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("✅ All libraries imported successfully")


✅ All libraries imported successfully


In [2]:
# ============================================================================
# CELL 2: Load Data and Preprocessing
# ============================================================================
print("\n" + "="*70)
print("LOADING DATA AND PREPROCESSING")
print("="*70)

db_path = '/Users/karthika/housing_app_fall25/db/housing_classification.db'
conn = sqlite3.connect(db_path)
df = pd.read_sql("SELECT * FROM housing_data", conn)
conn.close()

X = df.drop('PriceCategory', axis=1)
y = df['PriceCategory']

models_path = Path('/Users/karthika/housing_app_fall25/api/models')

# Load encoders
with open(models_path / 'label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Encode categorical variables
categorical_cols = X.select_dtypes(include=['object']).columns
for col in categorical_cols:
    X[col] = label_encoders[col].transform(X[col].astype(str))

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale
with open(models_path / 'scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Load PCA
with open(models_path / 'pca.pkl', 'rb') as f:
    pca = pickle.load(f)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"✅ Data loaded and preprocessed")
print(f"   Train shape: {X_train_scaled.shape}")
print(f"   Test shape: {X_test_scaled.shape}")
print(f"   PCA shape: {X_train_pca.shape}")

results = []


LOADING DATA AND PREPROCESSING
✅ Data loaded and preprocessed
   Train shape: (2344, 79)
   Test shape: (586, 79)
   PCA shape: (2344, 61)


In [3]:
# ============================================================================
# CELL 3: RandomForest - No PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 1/8: RandomForest + No PCA + Optuna")
print("="*70)

def objective_rf_nopca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'random_state': 42,
        'n_jobs': -1
    }
    model = RandomForestClassifier(**params)
    score = cross_val_score(model, X_train_scaled, y_train, cv=3, 
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_rf_nopca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_rf = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1)
best_rf.fit(X_train_scaled, y_train)
y_pred = best_rf.predict(X_test_scaled)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'RandomForest_NoPCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_rf, f)

results.append({
    'experiment_name': 'RandomForest_NoPCA_Optuna',
    'model_type': 'RandomForest',
    'pca': False,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 1/8: RandomForest + No PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Best CV F1 Score: 0.7779
✅ Test F1 Score: 0.8203



In [4]:
# ============================================================================
# CELL 4: GradientBoosting - No PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 2/8: GradientBoosting + No PCA + Optuna")
print("="*70)

def objective_gb_nopca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'random_state': 42
    }
    model = GradientBoostingClassifier(**params)
    score = cross_val_score(model, X_train_scaled, y_train, cv=3,
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_gb_nopca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_gb = GradientBoostingClassifier(**study.best_params, random_state=42)
best_gb.fit(X_train_scaled, y_train)
y_pred = best_gb.predict(X_test_scaled)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'GradientBoosting_NoPCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_gb, f)

results.append({
    'experiment_name': 'GradientBoosting_NoPCA_Optuna',
    'model_type': 'GradientBoosting',
    'pca': False,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 2/8: GradientBoosting + No PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Best CV F1 Score: 0.7797
✅ Test F1 Score: 0.7942



In [5]:
# ============================================================================
# CELL 5: XGBoost - No PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 3/8: XGBoost + No PCA + Optuna")
print("="*70)

def objective_xgb_nopca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42,
        'eval_metric': 'mlogloss',
        'use_label_encoder': False,
        'n_jobs': -1
    }
    model = XGBClassifier(**params)
    score = cross_val_score(model, X_train_scaled, y_train, cv=3,
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_xgb_nopca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_xgb = XGBClassifier(**study.best_params, random_state=42, eval_metric='mlogloss', 
                         use_label_encoder=False, n_jobs=-1)
best_xgb.fit(X_train_scaled, y_train)
y_pred = best_xgb.predict(X_test_scaled)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'XGBoost_NoPCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_xgb, f)

results.append({
    'experiment_name': 'XGBoost_NoPCA_Optuna',
    'model_type': 'XGBoost',
    'pca': False,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 3/8: XGBoost + No PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:44:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:44:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:44:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:44:37] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fo

✅ Best CV F1 Score: 0.7908
✅ Test F1 Score: 0.8188



In [6]:
# ============================================================================
# CELL 6: LightGBM - No PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 4/8: LightGBM + No PCA + Optuna")
print("="*70)

def objective_lgb_nopca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42,
        'verbose': -1,
        'n_jobs': -1
    }
    model = LGBMClassifier(**params)
    score = cross_val_score(model, X_train_scaled, y_train, cv=3,
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_lgb_nopca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_lgb = LGBMClassifier(**study.best_params, random_state=42, verbose=-1, n_jobs=-1)
best_lgb.fit(X_train_scaled, y_train)
y_pred = best_lgb.predict(X_test_scaled)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'LightGBM_NoPCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_lgb, f)

results.append({
    'experiment_name': 'LightGBM_NoPCA_Optuna',
    'model_type': 'LightGBM',
    'pca': False,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 4/8: LightGBM + No PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-p

✅ Best CV F1 Score: 0.7827
✅ Test F1 Score: 0.8005



In [7]:
# ============================================================================
# CELL 7: RandomForest - PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 5/8: RandomForest + PCA + Optuna")
print("="*70)

def objective_rf_pca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'random_state': 42,
        'n_jobs': -1
    }
    model = RandomForestClassifier(**params)
    score = cross_val_score(model, X_train_pca, y_train, cv=3,
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_rf_pca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_rf_pca = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1)
best_rf_pca.fit(X_train_pca, y_train)
y_pred = best_rf_pca.predict(X_test_pca)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'RandomForest_PCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_rf_pca, f)

results.append({
    'experiment_name': 'RandomForest_PCA_Optuna',
    'model_type': 'RandomForest',
    'pca': True,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 5/8: RandomForest + PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Best CV F1 Score: 0.7482
✅ Test F1 Score: 0.7618



In [8]:
# ============================================================================
# CELL 8: GradientBoosting - PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 6/8: GradientBoosting + PCA + Optuna")
print("="*70)

def objective_gb_pca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'random_state': 42
    }
    model = GradientBoostingClassifier(**params)
    score = cross_val_score(model, X_train_pca, y_train, cv=3,
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_gb_pca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_gb_pca = GradientBoostingClassifier(**study.best_params, random_state=42)
best_gb_pca.fit(X_train_pca, y_train)
y_pred = best_gb_pca.predict(X_test_pca)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'GradientBoosting_PCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_gb_pca, f)

results.append({
    'experiment_name': 'GradientBoosting_PCA_Optuna',
    'model_type': 'GradientBoosting',
    'pca': True,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 6/8: GradientBoosting + PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

✅ Best CV F1 Score: 0.7541
✅ Test F1 Score: 0.7794



In [9]:
# ============================================================================
# CELL 9: XGBoost - PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 7/8: XGBoost + PCA + Optuna")
print("="*70)

def objective_xgb_pca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42,
        'eval_metric': 'mlogloss',
        'use_label_encoder': False,
        'n_jobs': -1
    }
    model = XGBClassifier(**params)
    score = cross_val_score(model, X_train_pca, y_train, cv=3,
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_xgb_pca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_xgb_pca = XGBClassifier(**study.best_params, random_state=42, eval_metric='mlogloss',
                             use_label_encoder=False, n_jobs=-1)
best_xgb_pca.fit(X_train_pca, y_train)
y_pred = best_xgb_pca.predict(X_test_pca)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'XGBoost_PCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_xgb_pca, f)

results.append({
    'experiment_name': 'XGBoost_PCA_Optuna',
    'model_type': 'XGBoost',
    'pca': True,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 7/8: XGBoost + PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:54:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:54:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:54:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [13:54:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fo

✅ Best CV F1 Score: 0.7658
✅ Test F1 Score: 0.7804



In [10]:
# ============================================================================
# CELL 10: LightGBM - PCA + Optuna
# ============================================================================
print("\n" + "="*70)
print("EXPERIMENT 8/8: LightGBM + PCA + Optuna")
print("="*70)

def objective_lgb_pca(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42,
        'verbose': -1,
        'n_jobs': -1
    }
    model = LGBMClassifier(**params)
    score = cross_val_score(model, X_train_pca, y_train, cv=3,
                           scoring='f1_weighted', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective_lgb_pca, n_trials=20, show_progress_bar=True)

print(f"✅ Best CV F1 Score: {study.best_value:.4f}")

best_lgb_pca = LGBMClassifier(**study.best_params, random_state=42, verbose=-1, n_jobs=-1)
best_lgb_pca.fit(X_train_pca, y_train)
y_pred = best_lgb_pca.predict(X_test_pca)
f1 = f1_score(y_test, y_pred, average='weighted')

with open(models_path / 'LightGBM_PCA_Optuna.pkl', 'wb') as f:
    pickle.dump(best_lgb_pca, f)

results.append({
    'experiment_name': 'LightGBM_PCA_Optuna',
    'model_type': 'LightGBM',
    'pca': True,
    'optuna': True,
    'f1_score': f1
})

print(f"✅ Test F1 Score: {f1:.4f}\n")



EXPERIMENT 8/8: LightGBM + PCA + Optuna


  0%|          | 0/20 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-p

✅ Best CV F1 Score: 0.7682
✅ Test F1 Score: 0.7812



In [11]:
# ============================================================================
# CELL 11: Combine and Save All Results
# ============================================================================
print("\n" + "="*70)
print("COMBINING ALL RESULTS")
print("="*70)

results_df_part2 = pd.DataFrame(results)

# Load part 1 results
results_part1 = pd.read_csv(models_path / 'experiment_results_part1.csv')

# Combine all results
all_results = pd.concat([results_part1, results_df_part2], ignore_index=True)
all_results = all_results.sort_values('f1_score', ascending=False).reset_index(drop=True)
all_results.to_csv(models_path / 'experiment_results.csv', index=False)

print("\n" + "="*70)
print("🎉 ALL 16 EXPERIMENTS COMPLETED!")
print("="*70)
print(all_results.to_string(index=False))
print(f"\n✅ Final results saved to: {models_path / 'experiment_results.csv'}")
print(f"\n🏆 BEST MODEL: {all_results.loc[0, 'experiment_name']}")
print(f"   F1 Score: {all_results.loc[0, 'f1_score']:.4f}")
print(f"   Model Type: {all_results.loc[0, 'model_type']}")
print(f"   PCA: {all_results.loc[0, 'pca']}")
print(f"   Optuna: {all_results.loc[0, 'optuna']}")

print("\n" + "="*70)
print("✅ MODEL TRAINING COMPLETE!")
print("="*70)



COMBINING ALL RESULTS

🎉 ALL 16 EXPERIMENTS COMPLETED!
                experiment_name       model_type   pca  optuna  f1_score
      RandomForest_NoPCA_Optuna     RandomForest False    True  0.820311
           XGBoost_NoPCA_Optuna          XGBoost False    True  0.818759
    RandomForest_NoPCA_NoOptuna     RandomForest False   False  0.807307
        LightGBM_NoPCA_NoOptuna         LightGBM False   False  0.800565
          LightGBM_NoPCA_Optuna         LightGBM False    True  0.800457
         XGBoost_NoPCA_NoOptuna          XGBoost False   False  0.800107
GradientBoosting_NoPCA_NoOptuna GradientBoosting False   False  0.795696
  GradientBoosting_NoPCA_Optuna GradientBoosting False    True  0.794178
          LightGBM_PCA_NoOptuna         LightGBM  True   False  0.782840
            LightGBM_PCA_Optuna         LightGBM  True    True  0.781237
             XGBoost_PCA_Optuna          XGBoost  True    True  0.780407
    GradientBoosting_PCA_Optuna GradientBoosting  True    True  0.77